In [1]:
%load_ext autoreload

In [2]:
import torch 
import numpy as np
import random

from pixel_shuffle import PixelShuffle, PixelUnshuffle

%autoreload 2

In [3]:
batch_size = 3

In [4]:
# sanity check 
for _ in range(10):
    for n in range(1, 6):
        ps = PixelShuffle(n)
        pu = PixelUnshuffle(n)
        for k in range(1, 6):
            inputs = torch.randn(batch_size, n**k, *np.random.choice(range(1, 10), size=k))
            shuffled = ps(inputs)
            unshuffled = pu(shuffled)
            assert np.all(inputs.numpy() == unshuffled.numpy())

In [5]:
# torch_test
for _ in range(10):
    for n in range(1, 6):
        ps = PixelShuffle(n)
        pu = PixelUnshuffle(n)
        k = 2
        inputs = torch.randn(batch_size, n**k, *np.random.choice(range(1, 10), size=k))

        shuffled = ps(inputs)
        unshuffled = pu(shuffled)

        torch_shuffled = torch.nn.functional.pixel_shuffle(inputs, n)
        torch_unshuffled = torch.nn.functional.pixel_unshuffle(torch_shuffled, n)

        assert np.all(shuffled.numpy() == torch_shuffled.numpy()) 
        assert np.all(unshuffled.numpy() == torch_unshuffled.numpy()) 